## 掛載雲端硬碟

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

## 更改檔案所在路徑


In [16]:
# Change to your own folder !!!
# %cd /content/drive/MyDrive/your own folder/

## 載入函式庫


In [17]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np

from models.resnet import ResNet50

## 超參數設定

In [18]:
DATASET = 'cifar10'
TEST_BATCH_SIZE = 1000
CUDA = True
PRUNE_PERCENT = 0.9 # Change your prune ratio!
WEIGHT_PATH = 'model_best.pth' # Change to your own folder !!!
PRUNE_PATH = 'model_prune.pth' # Change to your own folder !!!

## 載入模型

In [19]:
import sys
print(sys.executable)

!pip show torch | grep Location
!python -c "import torch; print(torch.__version__)"
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
x = torch.randn(5, 5).cuda()
print("✅ CUDA OK:", x.device, x.sum().item())


/home/n26141826/Lab 2 - Model Pruning/.venv/bin/python
Location: /home/n26141826/Lab 2 - Model Pruning/.venv/lib/python3.12/site-packages
2.6.0+cu124
2.6.0+cu124
12.4
True
✅ CUDA OK: cuda:0 -3.776561737060547


In [20]:
CUDA = CUDA and torch.cuda.is_available()

model = ResNet50(num_classes=10)
for name, param in model.named_parameters():
    if param.numel() == 0:
        print(f"[ERROR] Empty tensor detected at {name}, shape={param.shape}")

if CUDA:
    model.cuda()

if WEIGHT_PATH:
    if os.path.isfile(WEIGHT_PATH):
        checkpoint = torch.load(WEIGHT_PATH)
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print('LOADING CHECKPOINT {} @EPOCH={}, BEST_PREC1={}'.format(WEIGHT_PATH,checkpoint['epoch'],best_prec1))

    else:
        print("NO CHECKPOINT FOUND")

print(model)

LOADING CHECKPOINT model_best.pth @EPOCH=38, BEST_PREC1=0.911
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential

## 進行剪枝
#### 計算所有Batch Normalizaiton中的scale factor絕對值大小並排序
#### 利用設定好的PRUNE_PERCENT來取得閥值

In [21]:
total = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        total += m.weight.data.shape[0]

bn = torch.zeros(total)
index = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        size = m.weight.data.shape[0]
        bn[index:(index+size)] = m.weight.data.abs().clone()
        index += size

y, i = torch.sort(bn)

threshold_index = int(total * PRUNE_PERCENT)
threshold = y[threshold_index]


## 根據Batch Normalization Layer資訊建立CONFIG
#### 1. 複製Batch Normalization Layer的weight(也就是scale factor γ)
#### 2. 建立mask，大於threshold的index的值會設成1,小於threshold的值會設成0
#### 3. mask的值加總後，會是剪枝後Layer對應的輸出channel數
#### 4. 最後得到要建立剪枝模型的CONFIG

In [22]:
pruned = 0
cfg = []  #用來建立剪枝網路的CONFIG
cfg_mask = [] #用來幫助剪枝的遮罩

In [23]:
for k, m in enumerate((model.modules())):
    if isinstance(m, nn.BatchNorm2d):
        weight_copy = m.weight.data.clone()
        mask = weight_copy.abs().gt(threshold).float().cuda() # 大於 threshold 的設為 True (1.0)，其餘為 False(0.0)

        # 注意: 需自行設計處理剩下channel數為0的情況 (e.g. 至少保留3個channel)
        ################################################
        #          請填空          #
        ################################################
        if int(torch.sum(mask)) == 0:
            print(f"Warning: layer {k} has 0 channels after pruning. Keeping 3.")
            mask[:3] = 1.0
        elif int(torch.sum(mask)) < 3:
            print(f"Layer {k} has very few channels ({int(torch.sum(mask))}), keeping 3 minimum.")
            # 強制保留最大的 3 個 gamma 值對應的 channel
            _, idx = torch.topk(weight_copy.abs(), 3)
            mask[idx] = 1.0



        # 處理剪枝後的權重
        m.weight.data.mul_(mask)
        m.bias.data.mul_(mask)
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        cfg.append(int(torch.sum(mask)))    # 記錄每一層 BN 剩下幾個通道
        cfg_mask.append(mask.clone())     # 儲存每層對應的 mask
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))

pruned_ratio = pruned/total

print(f'PRUNE RATIO={pruned_ratio}')
print('PREPROCESSING SUCCESSFUL!')

print(f'cfg: {cfg}')


layer index: 2 	 total channel: 64 	 remaining channel: 64
layer index: 8 	 total channel: 64 	 remaining channel: 64
layer index: 10 	 total channel: 64 	 remaining channel: 64
layer index: 12 	 total channel: 256 	 remaining channel: 239
layer index: 16 	 total channel: 256 	 remaining channel: 251
layer index: 19 	 total channel: 64 	 remaining channel: 64
layer index: 21 	 total channel: 64 	 remaining channel: 64
layer index: 23 	 total channel: 256 	 remaining channel: 174
layer index: 27 	 total channel: 64 	 remaining channel: 64
layer index: 29 	 total channel: 64 	 remaining channel: 64
layer index: 31 	 total channel: 256 	 remaining channel: 59
layer index: 36 	 total channel: 128 	 remaining channel: 120
layer index: 38 	 total channel: 128 	 remaining channel: 123
layer index: 40 	 total channel: 512 	 remaining channel: 71
layer index: 44 	 total channel: 512 	 remaining channel: 114
layer index: 47 	 total channel: 128 	 remaining channel: 93
layer index: 49 	 total cha

## 建立剪枝模型

In [24]:
newmodel = ResNet50(num_classes=10, cfg=cfg)
newmodel.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 239, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(239, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 239, kernel_size=(1, 1), stride=(1, 

### 將原本的模型權重複製到剪枝的模型
#### 根據不同層決定要複製什麼權重
###### Batch Normalization Layer
1.   scale factor
2.   bias
3.   running mean
4.   running variance

###### Convolutional Layer
1.   weight

###### Linear Layer
1.   weight
2.   bias




In [25]:
for i, m in enumerate(cfg_mask):
    try:
        m_cpu = m.detach().cpu()
        print(f"{i}: shape={tuple(m_cpu.shape)}, sum={m_cpu.sum().item()}, dtype={m_cpu.dtype}")
    except Exception as e:
        print(f"[ERROR] Layer {i} failed: {e}")


0: shape=(64,), sum=64.0, dtype=torch.float32
1: shape=(64,), sum=64.0, dtype=torch.float32
2: shape=(64,), sum=64.0, dtype=torch.float32
3: shape=(256,), sum=239.0, dtype=torch.float32
4: shape=(256,), sum=251.0, dtype=torch.float32
5: shape=(64,), sum=64.0, dtype=torch.float32
6: shape=(64,), sum=64.0, dtype=torch.float32
7: shape=(256,), sum=174.0, dtype=torch.float32
8: shape=(64,), sum=64.0, dtype=torch.float32
9: shape=(64,), sum=64.0, dtype=torch.float32
10: shape=(256,), sum=59.0, dtype=torch.float32
11: shape=(128,), sum=120.0, dtype=torch.float32
12: shape=(128,), sum=123.0, dtype=torch.float32
13: shape=(512,), sum=71.0, dtype=torch.float32
14: shape=(512,), sum=114.0, dtype=torch.float32
15: shape=(128,), sum=93.0, dtype=torch.float32
16: shape=(128,), sum=121.0, dtype=torch.float32
17: shape=(512,), sum=18.0, dtype=torch.float32
18: shape=(128,), sum=98.0, dtype=torch.float32
19: shape=(128,), sum=115.0, dtype=torch.float32
20: shape=(512,), sum=15.0, dtype=torch.float32
2

In [26]:
old_modules = list(model.modules())
new_modules = list(newmodel.modules())

layer_id_in_cfg = 0
start_mask = torch.ones(3) #3為input channel(R,G,B)
end_mask = cfg_mask[layer_id_in_cfg]
bn_count = 0

for layer_id in range(len(old_modules)):

    m0 = old_modules[layer_id]
    m1 = new_modules[layer_id]

    if isinstance(m0, nn.BatchNorm2d):
        bn_count += 1

        #### 找出遮罩中非零元素的index ####
        ################################################
        #          請填空          #
        ################################################
        idx = np.arange(m0.weight.data.shape[0])  # 不再根據mask縮減index


        #### 複製weight, bias, running mean,and running variance ####
        ################################################
        #          請填空          #
        ################################################
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
        m1.running_mean = m0.running_mean.clone()
        m1.running_var = m0.running_var.clone()
        # 加入mask乘法，讓被剪掉的通道實際為0，但不改結構
        m1.weight.data.mul_(cfg_mask[layer_id_in_cfg])
        m1.bias.data.mul_(cfg_mask[layer_id_in_cfg])

        # 但我們仍要保留 mask（代表稀疏化通道）
        # 可以用 mask 做為 fine-tune 或 zeroed-weight 統計
        m1.weight.data.mul_(cfg_mask[layer_id_in_cfg])
        m1.bias.data.mul_(cfg_mask[layer_id_in_cfg])


        # 更新 mask
        layer_id_in_cfg += 1
        start_mask = end_mask.clone()

        #最後一層連接層不做修改
        if layer_id_in_cfg < len(cfg_mask):
            end_mask = cfg_mask[layer_id_in_cfg]


    elif isinstance(m0, nn.Conv2d):
        if isinstance(old_modules[layer_id + 1], nn.BatchNorm2d):
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().detach().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().detach().numpy())))
            if len(idx1) == 0: idx1 = np.arange(m0.weight.data.shape[0])
            if len(idx0) == 0: idx0 = np.arange(m0.weight.data.shape[1])


            #### 複製weight ####
            ################################################
            #          請填空          #
            ################################################
            m1.weight.data = m0.weight.data.clone()
            if m0.bias is not None:
                m1.bias.data = m0.bias.data.clone()


        # downsample 層不用prune
        else:
            m1.weight.data = m0.weight.data.clone()


    elif isinstance(m0, nn.Linear):

        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        if idx0.ndim == 0:
            idx0 = np.expand_dims(idx0, 0)

        #### 複製weight ####
        ################################################
        #          請填空          #
        ################################################
        m1.weight.data = m0.weight.data.clone()

        #### 複製bias ####
        m1.bias.data = m0.bias.data.clone()


## 測試函數




In [27]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
        batch_size=TEST_BATCH_SIZE, shuffle=True, **kwargs)

    model.eval()
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
          if CUDA:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

## 儲存模型並印出結果，以及剪枝後的test acc


In [28]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()}, PRUNE_PATH)

print(newmodel)
model = newmodel.cuda()
test(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 239, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(239, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 239, kernel_size=(1, 1), stride=(1, 

tensor(0.1383)